In [ ]:
%matplotlib inline
import nupack, numpy as np, matplotlib.pyplot as plt
from scipy.integrate import odeint, OdeSolver
import numpy.linalg as la

In [ ]:
from local_time import equilibrium_info, generate_locs, structure_matrix_from_state, Z_from_R, Gamma_path_estimate

In [ ]:
model = nupack.Model(gt = True)

w = nupack.State('GCGUCGCGUCGCUAUGC',dp = '.....((((....))))', kind='full', model=model) # from Brian's paper

w_unpaired = nupack.State('GCGUCGCGUCGCUAUGC', kind='full', model=model) # unpaired for use with rate matrix
# self = sequence, strands = can be list of strands in state, dp = dot parens (default is unpaired),
# kind = move generation algorithm (lazy, full or static, default is lazy), model = energy model to use

# print(w.dp())
# print(w.sequence())
# print(w.pairs.view())
# print(w)

states, pi, R, _ = w_unpaired.rate_matrix()
R = R.toarray()

In [ ]:
# use Analysis to sample a set of structures based on the Boltzmann distribution
def equilibrium_info(n, sequences = ('GCGUCGCGUCGCUAUGC',)):
    analysis = nupack.Analysis(model=model)
    analysis.boltzmann_sample(sequences, n) # generates 1e6 (or set #) of sample structures within the boltzmann distribution

    analysis.pair_probability(sequences)
    results = analysis.compute() # compute partition function, MFE, PP matrix, etc. for each sequence in boltzmann sample
    pairs = results[sequences].pair_probability
    free_energy = results[sequences].free_energy
    samples = [s.dp() for s in results[sequences].boltzmann_sample]  # return dot-parens for each structure in sample
    return pairs, samples, free_energy

In [ ]:
def generate_locs(pairs, n=3):
    variance = np.triu(pairs * (1 - pairs))
    threshold = np.sort(variance.ravel())[-n]
    rows, cols = np.where(variance >= threshold)
    ordered = rows <= cols
    rows = rows[ordered]
    cols = cols[ordered]
    locs = rows, cols
    list(zip(*locs))
    return locs

In [ ]:
# list(zip(*locs))

In [ ]:
# rows, cols = np.where((pairs > 1e-1) & (pairs < 0.99))
# ordered = rows <= cols
# rows = rows[ordered]
# cols = cols[ordered]
# locs = rows, cols

In [ ]:
# structure matrix function where 1's indicate paired bases (1's on diag indicate unpaired)

def structure_matrix_from_state(w):
    pairs = w.pairs.view()
    F = np.zeros((len(pairs), len(pairs)))
    for i, j in enumerate(pairs):
        F[i, j] = 1
    return F[1:-1, 1:-1]

In [ ]:
type(pairs[locs])
type(avg_F)

In [ ]:
np.diag(pi) @ np.ones([779,779]) @ np.diag(pi)

In [ ]:
# pi_analytic = np.tile(reduced_pi, (3,1)) # square matrix of stationary values
# define a function to calculate Z
def Z_from_R(pi, R):
    print(np.diagonal(R).sum())
    return np.linalg.inv(-R + 1e6 * pi[None]) - 1e-6 * pi[None]    # pi[None] creates square matrix of pi values

S = nupack.markov.S_from_R(R, pi)

# bigF = np.array([structure_matrix_from_state(w)[locs] for w in states])
bigF = np.array([[w.pairs == w_mfe.pairs for w_mfe in mfe_states] for w in states])

Gamma = bigF.T @ (np.diag(pi) @ S @ np.diag(pi)) @ bigF

In [ ]:
Gamma

In [ ]:
C = bigF.T @ np.diag(pi) @ bigF # gives strength of correlation between base pairs in basis function

In [ ]:
Gamma.shape

In [ ]:
# tau_range = np.array([1e-08, 2e-08, 1e-07, 2e-07], dtype = float)
tau_range = [1e-08]
for n in range(18):
    tau_range.append(tau_range[-1]*2)
# print(tau_range)
# np.array(tau_range)

In [ ]:
bigF = np.array([structure_matrix_from_state(w)[locs] for w in states])

In [ ]:
## Turn this into a function with input arguments: n (# runs), # samples, sequences, tau_range, trajectory #'s to test, 

# run a series of sample path trajectories for set tau using sample_path

def Gamma_path_estimate(tau_range, sequences, runs = 1, sample_num = 1e4, traj_num = 10000):

    Gamma_error = []
    Gamma_values = []
    Gamma_actual = []
    
    w_unpaired = nupack.State(sequences, kind='full', model=model)
    states, pi, R, _ = w_unpaired.rate_matrix()

    S = nupack.markov.S_from_R(R, pi)

    for n in range(runs):

        pairs, samples, free_energy = equilibrium_info(sample_num, sequences) # generate new sample distribution each time
        locs = generate_locs(pairs)


        bigF = np.array([structure_matrix_from_state(w)[locs] for w in states])

        Gamma = bigF.T @ (np.diag(pi) @ S @ np.diag(pi)) @ bigF

        avgF = pairs[locs] # F will approach equilibrium probabilitiy distribution

        Gamma_error_run = []
        Gamma_values_run = []
        starts = [nupack.State(sequences[0], dp=dp) for dp in samples] # generate starting states from dp of structures in samples

        # Check whether tau_range is a list or one value
        if (len(np.array(tau_range)) == 1):

            paths = [nupack.path.sample_path(w, tau_range[0]) for w in starts[:traj_num]]

            # 'times' is the integrated pair probability matrix for the sample path, i.e. the amount of time a base pair exists during the trajectory
            # since times is the integrated PP, have to multiply mean PP (avg_F) by time to have equivalent units. Subtract off so that can calculate covariance from mean value
            times = np.array([p.base_pair_time()[locs] - p.time * avgF for p in paths]) 
            total_time = sum(p.time for p in paths)

            G_estimate = times.T @ times / 2 / total_time  # transpose turns columns into rows, then times columns of times, means multiplying each element by itself
            Gamma_error_run.append((np.linalg.norm(G_estimate - Gamma) / np.linalg.norm(Gamma)))
            Gamma_values_run.append(G_estimate)

        else:
            for tau in enumerate(tau_range):

                paths = [nupack.path.sample_path(w, tau[1]) for w in starts[:traj_num]]

                times = np.array([p.base_pair_time()[locs] - p.time * avgF for p in paths]) 
                total_time = sum(p.time for p in paths)

                G_estimate = times.T @ times / 2 / total_time
                Gamma_error_run.append((np.linalg.norm(G_estimate - Gamma) / np.linalg.norm(Gamma)))
                Gamma_values_run.append(G_estimate)

        Gamma_error.append(Gamma_error_run)
        Gamma_values.append(Gamma_values_run)
        Gamma_actual.append(Gamma)

        print(n)

    return Gamma_actual, Gamma_values, Gamma_error

In [ ]:
Gamma_actual, Gamma_values, Gamma_error = Gamma_path_estimate(tau_range, sequences = ('GCGUCGCGUCGCUAUGC',), runs = 10)
np.array(Gamma_error).shape

In [ ]:
Gamma_error = np.array(Gamma_error)
Gamma_values = np.array(Gamma_values)

print(Gamma_error.shape)
print(G_estimate.shape)
print(Gamma_values.shape)
print(Gamma_values[0][0].shape)
print(Gamma_values[0][0].shape)
(np.linalg.norm(G_estimate - Gamma) / np.linalg.norm(Gamma))

In [ ]:


def corrected_gamma(G1,G2):
    '''
    Returns G1 @ inv(2 * G1 - G2) @ G1
    '''
    return G1 @ la.inv(2 * G1 - G2) @ G1
    evals, evecs = la.eigh(G1)
    proj = (evecs * ((2 * G1 - G2) @ evecs).sum(1))
    # make sure timescale estimate is only increased
#     evals = np.maximum(evals,evals**2/proj)
    evals = evals**2/proj
    return evecs @ np.diag(evals) @ evecs.T

In [ ]:
# second Gamma estimate based on Gamma(tau) * (2*Gamma(tau) - 0.5*Gamma(2*tau))^(-1) * Gamma(tau)

Gamma_error_other = []
for j in range(Gamma_values.shape[0]):
    G_estimate_other_run = []
    Gamma_error_other_run = []
    Gamma_for_run = Gamma_values[j]
# Gamma_for_run = Gamma_values
    for n in range(len(tau_range)-1):
        Gamma_tau = Gamma_for_run[0][n]
        Gamma_2tau = Gamma_for_run[0][n+1] 
#         print(n, np.linalg.eigvals(Gamma_tau - 0 * Gamma_2tau).min())
        G_estimate_other_run = corrected_gamma(Gamma_tau, Gamma_2tau)
#         G_estimate_other_run = Gamma_tau @ np.linalg.pinv(2 * Gamma_tau - Gamma_2tau) @ Gamma_tau
        Gamma_error_other_run.append(np.linalg.norm(G_estimate_other_run - Gamma) / np.linalg.norm(Gamma))
    #     G_estimate_other = Gamma_tau @ np.linalg.pinv(2 * Gamma_tau - Gamma_2tau) @ Gamma_tau
    #     Gamma_error_other.append(np.linalg.norm(G_estimate_other - Gamma) / np.linalg.norm(Gamma))
    Gamma_error_other.append(Gamma_error_other_run)
    #     print(j)
    #     print(Gamma_error_other_run)

print(np.array(Gamma_error_other).shape)

In [ ]:
Gamma_error_other = np.array(Gamma_error_other)

In [ ]:
# Calculate mean and standard deviation of Gamma Error from 10 runs
Gamma_error_mean = np.mean(Gamma_error, axis = 0)
Gamma_error_std = np.std(Gamma_error, axis = 0)
print('Mean Gamma Error = ', Gamma_error_mean)
print('Standard Deviation of Gamma = ', Gamma_error_std)

In [ ]:
# Calculate mean and standard deviation of Second Gamma Method Error from 10 runs
Gamma_error_other_mean = np.mean(Gamma_error_other, axis = 0)
Gamma_error_other_std = np.std(Gamma_error_other, axis = 0)
print('Mean Gamma = ', Gamma_error_other_mean)
print('Standard Deviation of Gamma = ', Gamma_error_other_std)

In [ ]:
la.eig?

In [ ]:
# Calculate Analytical estimate of gamma

eigvals, eigvecs = la.eig(R)
R2 = eigvecs @ np.diag(eigvals) @ la.inv(eigvecs)
print(np.abs(R - R2).mean())



G0 = np.diag(pi) @ S @ np.diag(pi)

def truncated_gamma(T):
    print(T)
    order = np.argsort(eigvals)
    eigvals2 = eigvals[order].astype(np.float128)
    eigvecs2 = eigvecs[:, order]
#     eigvals2[-1] = 0
    eigvals2[:-1] = (np.exp(T * eigvals2[:-1]) - 1) / eigvals2[:-1]
    return np.diag(pi) @ eigvecs2 @ np.diag(eigvals2.astype(np.float64)) @ la.inv(eigvecs2)

Ts = 10**np.linspace(-8, -2, 20)
results = [1 - la.norm(truncated_gamma(T) - G0) / la.norm(G0) for T in Ts]

plt.plot(Ts, results, marker = '.')

plt.xscale('log')
plt.yscale('log')
plt.xlabel('time (s)')
plt.ylabel('1 - Gamma Error')
plt.title('Convergence of Truncated Analytical Gamma')
# eigen_analyt = eigen_analyt[np.argsort(-eigen_analyt)]
# eigen_analyt.shape
# T = 0.004
# Gamma_T = []
# T_range = np.array([0, 0.004])
# for T in T_range:
#     Gamma_T_values = [((1 - np.exp(eigen_analyt[n + 1] * T)) / eigen_analyt[n + 1]) for n in range(len(eigen_analyt) - 1)]
#     Gamma_T.append(np.array(Gamma_T_values).sum())
# Gamma_T = np.array(Gamma_T)
# # np.array(Gamma_T_values).sum()
# Gamma_analyt = la.norm(bigF.T @ Gamma_T[1] @ bigF - bigF.T @ Gamma_T[0] @ bigF)
# # Gamma_0 = 

In [ ]:
plt.plot(np.sort(la.eigvals(G0))[::-1][:10], marker='.')
# plt.yscale('log')

In [ ]:
# Non-negative Matrix Factorization of Actual Gamma (G0) as example of how to get to rate matrix

from sklearn.decomposition import NMF

nmf_model = NMF(n_components=3)
W = nmf_model.fit_transform(np.maximum(0, G0))
H = nmf_model.components_
err = nmf_model.reconstruction_err_

In [ ]:
w = W.sum(0) * H.sum(1)
plt.plot(w, marker='.')

In [ ]:
F = W / W.sum(0)

In [ ]:
F.shape

In [ ]:
# Testing number of Trajectories for Gamma vs. Tau

# plt.plot(tau_range,1-Gamma_error_mean, label = 'Gamma Error by Path Estimate', marker='.')
# # plt.plot(tau_range[:-1],1-Gamma_error_other_mean, label = 'Gamma Error by Second Formula', marker='.')
# plt.plot(tau_range,1-Gamma_error[0,:], label = '10 trajectories', marker='.')
# plt.plot(tau_range,1-Gamma_error[1,:], label = '100 trajectories', marker='.')
plt.plot(tau_range,1-Gamma_error[0][0,:], label = '1000 trajectories', marker='.')
plt.plot(tau_range,1-Gamma_error[1][0,:], label = '10000 trajectories', marker='.')
plt.plot(tau_range,1-Gamma_error[2][0,:], label = '100000 trajectories', marker='.')
plt.plot(tau_range[:-1],1-Gamma_error_other[0,:], label = '1000 trajectories', marker='.')
plt.plot(tau_range[:-1],1-Gamma_error_other[1,:],color = 'r', label = '10000 trajectories', marker='.')
plt.plot(tau_range[:-1],1-Gamma_error_other[2,:],color = 'b', label = '100000 trajectories', marker='.')

plt.yscale('log')
plt.xscale('log')
plt.xlabel('log(tau)')
plt.ylabel('log(Gamma Error)')
# plt.title('Gamma Error by Path Estimate')
plt.legend()

In [ ]:
# Testing number of Trajectories for Gamma vs. Tau

plt.scatter(tau_range[:-1],1-Gamma_error_other[0,:], label = '10 trajectories', marker='.')
plt.scatter(tau_range[:-1],1-Gamma_error_other[1,:], label = '100 trajectories', marker='.')
plt.scatter(tau_range[:-1],1-Gamma_error_other[2,:], label = '1000 trajectories', marker='.')
plt.plot(tau_range[:-1],1-Gamma_error_other[3,:],color = 'r', label = '10000 trajectories', marker='.')
plt.plot(tau_range[:-1],1-Gamma_error_other[4,:],color = 'b', label = '100000 trajectories', marker='.')

plt.yscale('log')
plt.xscale('log')
plt.xlabel('log(tau)')
plt.ylabel('log(Gamma Error)')
# plt.title('Gamma Error by Second Formula Estimate')
plt.legend(loc = 'upper left')
# plt.legend.loc(2)

In [ ]:
# estimate of simulation time for one trajectory, length 1 second
%%time
nupack.path.sample_path(w, 1.0)

In [ ]:
# plt.plot(tau_range,Gamma_error_mean, label = 'Gamma Error by Path Estimate', marker='.')
# plt.plot(tau_range[:-1],Gamma_error_other_mean, label = 'Gamma Error by Second Formula', marker='.')
plt.errorbar(tau_range, 1 - Gamma_error_mean, yerr = Gamma_error_std, ecolor = 'k',color = '#1f77b4', label = 'Gamma Error by Path Estimate', marker='.')
plt.errorbar(tau_range[:-1], 1 - Gamma_error_other_mean, yerr = Gamma_error_other_std, ecolor = 'k',color = '#ff7f0e', label = 'Gamma Error by Second Formula', marker='.')
# plt.axis([0, 0.004, 0, 1])
plt.yscale('log')
plt.xscale('log')
plt.xlabel('log(tau)')
plt.ylabel('log(Gamma Error)')
plt.legend()



In [ ]:
# USING RATE MATRIX AS BASIS FUNCTION

In [ ]:
## Using MFE States in Basis Function to find R from Gamma

mfe_states = [nupack.State('GCGTCGCGTCGCTATGC', dp) for dp in ('.....((((....))))', '(((......))).....', '((((.((...)).))))')]
# path = nupack.path.sample_path(w, 0.01)
# times = path.structure_time([w.pairs for w in mfe_states])
# print(times)

avgF = np.array([model.boltz(e - free_energy) for e in (-1.5, -2.6, -4.1)])
# to get something comparable to the small-box (to get a rate matrix), use as the basis, a 1 corresponding to the presence of a particular microstate (MFE), then can calculate time-integrated probabilities of these microstates to get R

S = nupack.markov.S_from_R(R, pi)
w.pairs.view()

In [ ]:
# run a series of sample path trajectories for set tau using sample_path
# avgF = pairs[locs] # F will approach equilibrium probabilitiy distribution
sequences = ('GCGUCGCGUCGCUAUGC',)
avgF = np.array([model.boltz(e - free_energy) for e in (-1.5, -2.6, -4.1)])
bigF = np.array([[w.pairs == w_mfe.pairs for w_mfe in mfe_states] for w in states])
Gamma = bigF.T @ (np.diag(pi) @ S @ np.diag(pi)) @ bigF

Gamma_error = []
Gamma_values = []
Gamma_actual = []
# variance = []
# trajectory_range = [10, 100, 1000, 10000]
# for n in enumerate(trajectory_range):
for n in range(10):
    pairs, samples, free_energy = equilibrium_info(1e4) # generate new sample distribution each time
#     locs = generate_locs(pairs)
#     bigF = np.array([structure_matrix_from_state(w)[locs] for w in states])

    Gamma_error_run = []
    Gamma_values_run = []
    starts = [nupack.State(sequences[0], dp=dp) for dp in samples] # generate starting states from dp of structures in samples
    for tau in enumerate(tau_range):
#         paths = [nupack.path.sample_path(w, tau[1]) for w in starts[:n[1]]]
#         paths = [nupack.path.sample_path(w, tau[1]) for w in starts]
        paths = [nupack.path.sample_path(w, tau[1]) for w in starts]

        # 'times' is the integrated pair probability matrix for the sample path, i.e. the amount of time a base pair exists during the trajectory
#         times = np.array([p.base_pair_time()[locs] - p.time * avgF for p in paths]) # since times is the integrated PP, have to multiply mean PP (avg_F) by time to have equivalent units. Subtract off so that can calculate covariance from mean value.
        times = np.array([p.structure_time([w.pairs for w in mfe_states]) - p.time * avgF for p in paths])
        total_time = sum(p.time for p in paths)

        G_estimate = times.T @ times / 2 / total_time
        Gamma_error_run.append((np.linalg.norm(G_estimate - Gamma) / np.linalg.norm(Gamma)))

        ## Some other options for error calculations
#         Gamma_error = np.linalg.norm(G_estimate - Gamma) / np.amax(Gamma)
#         Gamma_error = np.trace(G_estimate) / np.trace(Gamma)
        
        Gamma_values_run.append(G_estimate)
#     Gamma_error_run.append((np.linalg.norm(G_estimate - Gamma) / np.linalg.norm(Gamma)))
#     Gamma_values.append(G_estimate)
        
    Gamma_error.append(Gamma_error_run)
    Gamma_values.append(Gamma_values_run)
    Gamma_actual.append(Gamma)
    print(n)

In [ ]:
Gamma_values = np.array(Gamma_values)
Gamma_values_mean = np.mean(Gamma_values, axis = 0)
Gamma_values_mean.shape
Gamma_values.shape

In [ ]:
# Use mean Gamma estimate values to calculate rate matrix

# average across 10 runs for each tau values
# Gamma_mean = np.mean(Gamma_values, axis = 0)
R_new = []
for n in range(len(tau_range)):
    G = Gamma_values_mean[n]

    mu = bigF.T @ pi
    N = mu.T @ la.inv(G) @ mu
    pi_new = (np.diag(mu) @ la.inv(G) @ mu) / N
    M = np.outer(mu, mu) / N
    Gamma_new = (np.eye(3) - la.inv(G) @ M).T @ G @ (np.eye(3) - la.inv(G) @ M)
    PI_new = np.outer(pi_new, pi_new) / (pi_new.T @ pi_new)
    S_new = la.inv(np.diag(mu)) @ Gamma_new @ la.inv(np.diag(mu))
    R_new.append(-(np.diag(1 / pi_new) @ la.inv(S_new + 1) - np.outer(np.ones(3),pi_new)))
# print('mu', mu)
# print((mu @ mu.T).shape, 'N', N)
# print(mu.shape, N.shape)
# print('pi_new', pi_new, pi_new.sum(), M)
# print(la.inv(G).shape)
# T = la.inv(Gamma_new + PI_new) - PI_new
# print(T.shape, la.inv(pi_squiggle))))
# la.inv(pi_squiggle)
# R_squiggle = -T @ np.diag(pi_squiggle)

R_new = np.array(R_new)
print(R_new.shape)
# print('S_new',S_new)
# print('R_new',R_new)

In [ ]:
R_analytic = np.array([[-5411.10410896,  3025.12682361,  2385.97728535],
       [  754.86048218, -2001.22603886,  1246.36555668],
       [   62.91242353,   131.70202294,  -194.61444647]])
# print(R_new / R_analytic)
R_new_error = [(la.norm(R_new[t] - R_analytic) / la.norm(R_analytic)) for t in range(len(tau_range))]
R_new_error = np.array(R_new_error)
print(R_new_error.shape)
print(R_new_error)

In [ ]:
# second Gamma estimate based on Gamma(tau) * (2*Gamma(tau) - 0.5*Gamma(2*tau))^(-1) * Gamma(tau)

Gamma_error_other = []
Gamma_values_other = []
for j in range(Gamma_values.shape[0]):
    G_estimate_other_run = []
    Gamma_error_other_run = []
    Gamma_for_run = Gamma_values[j]
# Gamma_for_run = Gamma_values
    for n in range(len(tau_range)-1):
        Gamma_tau = Gamma_for_run[n]
        Gamma_2tau = Gamma_for_run[n+1] 
#         print(n, np.linalg.eigvals(Gamma_tau - 0 * Gamma_2tau).min())
        G_estimate_other_run.append(corrected_gamma(Gamma_tau, Gamma_2tau))
#         Gamma_values_other_run.append(G_estimate_other_run)
#         G_estimate_other_run = Gamma_tau @ np.linalg.pinv(2 * Gamma_tau - Gamma_2tau) @ Gamma_tau
        Gamma_error_other_run.append(np.linalg.norm(G_estimate_other_run - Gamma) / np.linalg.norm(Gamma))
    #     G_estimate_other = Gamma_tau @ np.linalg.pinv(2 * Gamma_tau - Gamma_2tau) @ Gamma_tau
    #     Gamma_error_other.append(np.linalg.norm(G_estimate_other - Gamma) / np.linalg.norm(Gamma))
    Gamma_error_other.append(Gamma_error_other_run)
    Gamma_values_other.append(G_estimate_other_run)
    #     print(j)
    #     print(Gamma_error_other_run)

print(np.array(Gamma_values_other).shape)

In [ ]:
Gamma_values_other_mean = np.mean(Gamma_values_other, axis = 0)
Gamma_values_other_mean.shape

In [ ]:
# Use Corrected Mean Gamma estimate values to calculate rate matrix

# average across 10 runs for each tau values
# Gamma_mean = np.mean(Gamma_values, axis = 0)
R_new_corrected = []
for n in range(len(tau_range) - 1):
    G = Gamma_values_other_mean[n]

    mu = bigF.T @ pi
    N = mu.T @ la.inv(G) @ mu
    pi_new = (np.diag(mu) @ la.inv(G) @ mu) / N
    M = np.outer(mu, mu) / N
    Gamma_new = (np.eye(3) - la.inv(G) @ M).T @ G @ (np.eye(3) - la.inv(G) @ M)
    PI_new = np.outer(pi_new, pi_new) / (pi_new.T @ pi_new)
    S_new = la.inv(np.diag(mu)) @ Gamma_new @ la.inv(np.diag(mu))
    R_new_corrected.append(-(np.diag(1 / pi_new) @ la.inv(S_new + 1) - np.outer(np.ones(3),pi_new)))
# print('mu', mu)
# print((mu @ mu.T).shape, 'N', N)
# print(mu.shape, N.shape)
# print('pi_new', pi_new, pi_new.sum(), M)
# print(la.inv(G).shape)
# T = la.inv(Gamma_new + PI_new) - PI_new
# print(T.shape, la.inv(pi_squiggle))))
# la.inv(pi_squiggle)
# R_squiggle = -T @ np.diag(pi_squiggle)

R_new_corrected = np.array(R_new_corrected)
print(R_new_corrected.shape)

In [ ]:
R_new_corrected_error = [(la.norm(R_new_corrected[t] - R_analytic) / la.norm(R_analytic)) for t in range(len(tau_range) - 1)]
R_new_corrected_error = np.array(R_new_corrected_error)
print('Corrected Gamma',R_new_corrected[-1])
print('Not Corrected',R_new[-1])

In [ ]:
# print(1/np.linalg.eigh(G)[0])
# np.linalg.eigh(PI_squiggle)[0]
# pi_squiggle @ R_squiggle
# pi_squiggle @ T # np.linalg.eigh(Gamma_squiggle + PI_squiggle)[0]
# (np.linalg.eigh(Gamma_squiggle)[1][:, 0])/np.linalg.norm(np.linalg.eigh(Gamma_squiggle)[1][:, 0])
# pi_squiggle.sum()
# R_squiggle.sum(1)
# print(pi_squiggle/np.linalg.norm(pi_squiggle))
# np.linalg.eigh(T)[1][:, 0]

In [ ]:
# Testing number of Trajectories for Gamma vs. Tau

plt.plot(tau_range,R_new_error, label = 'Rate Matrix Error by Path Estimate', marker='.')
plt.plot(tau_range[:-1],R_new_corrected_error, label = 'Rate Matrix Error by Corrected Gamma', marker='.')

plt.yscale('log')
plt.xscale('log')
plt.xlabel('tau')
plt.ylabel('Normalized Rate Matrix Error)')
# plt.title('Gamma Error by Path Estimate')
plt.legend()

In [ ]:
# ODE's to be solved for enumerated solution

def differential_eqs(Prob, t, R):
    return Prob @ R

# start & stop times, initial state for solving ODE's

tspan = np.linspace(0, 4e-3,1000, dtype = float)

# Plug in reduced rate matrix
Gamma_Prob = odeint(differential_eqs,t=tspan,y0=np.array([1,0,0], dtype = float), args = (R_new[-1],)) 
Corrected_Gamma_Prob = odeint(differential_eqs,t=tspan,y0=np.array([1,0,0], dtype = float), args = (R_new_corrected[-1],)) 

In [ ]:
plt.plot(tspan,Gamma_Prob[:,0], label = 'Macrostate 1')
plt.plot(tspan,Gamma_Prob[:,1], label = 'Macrostate 2')
plt.plot(tspan,Gamma_Prob[:,2], label = 'Macrostate 3')
plt.axis([0, 0.004, 0, 1])
plt.xlabel('time(s)')
plt.ylabel('probability')
plt.legend()

In [ ]:
plt.plot(tspan,Corrected_Gamma_Prob[:,0], label = 'Macrostate 1')
plt.plot(tspan,Corrected_Gamma_Prob[:,1], label = 'Macrostate 2')
plt.plot(tspan,Corrected_Gamma_Prob[:,2], label = 'Macrostate 3')
plt.axis([0, 0.004, 0, 1])
plt.xlabel('time(s)')
plt.ylabel('probability')
plt.legend()

In [ ]:
# Numerical Solution by Matrix Exponentiation

t_init = 0
t_end = 0.004
t_step = (t_end - t_init) / 1000

P_i = np.array([[1], [0], [0]])
P = [np.array([[1], [0], [0]])]

from scipy.linalg import expm

t = t_init
t_log = [t_init]
while t <= t_end:
    P_new = expm(R_analytic * t).T @ P_i
    P.append(P_new)
#     P_i = P_new
    t += t_step
    t_log.append(t)

P = np.array(P)
P = P.reshape(1001,3)
t_log = np.array(t_log)

In [ ]:
plt.plot(t_log,P[:,0], label = 'Macrostate 1')
plt.plot(t_log,P[:,1], label = 'Macrostate 2')
plt.plot(t_log,P[:,2], label = 'Macrostate 3')
plt.axis([0, 0.004, 0, 1])
plt.xlabel('time(s)')
plt.ylabel('probability')
plt.legend()

In [ ]:
# Numerical Solution by Eigen-Expansion

t_range = np.linspace(0, 0.004,1000)

RH_eigvals, RH_eigvecs = la.eig(R_analytic.T)
pi_analytic = RH_eigvecs[:,2] / (RH_eigvecs[:,2].sum(0))
print(pi_analytic)
#

indices = (-RH_eigvals).argsort()
eigvals = RH_eigvals[indices]
eigvecs = RH_eigvecs[:,indices]
print('eigvals', eigvals)
print('eigvecs', eigvecs)

constants = la.inv(eigvecs) @ (P_i)


print('constants',constants)
print('check that this is pi_analytic: ',(constants[0] * eigvecs[:,0]))

P_eigen = [(constants[0] * eigvecs[:,0] + constants[1] * eigvecs[:,1] * (np.exp(eigvals[1] * t)) + constants[2] * eigvecs[:,2] * (np.exp(eigvals[2] * t))) for t in t_range]
P_eigen = np.array(P_eigen)

In [ ]:
plt.plot(t_range,P_eigen[:,0], label = 'Macrostate 1')
plt.plot(t_range,P_eigen[:,1], label = 'Macrostate 2')
plt.plot(t_range,P_eigen[:,2], label = 'Macrostate 3')
plt.axis([0, 0.004, 0, 1])
plt.xlabel('time(s)')
plt.ylabel('probability')
plt.legend()

In [ ]:
# # RAN 10 RUNS FOR ONE TAU VALUE AND STORED GAMMA AND VARIANCE
# # Need to calculate approximate "n" using n = O(tr(var)/(Tr(mean))**2)

# variance = np.array(variance)
# print(variance.shape, Gamma_values.shape)



# print(np.mean(Gamma_values, axis = 0))
# n = []
# n = np.mean(np.trace(variance[:])) / np.mean(np.trace(Gamma_values[:]))**2
# # for j in range(Gamma_values.shape[0]):
# #     n.append(np.trace(variance[j]) / np.trace(Gamma_values[j])**2)
# print(n)

In [ ]:
np.linalg.eigvals(Gamma)

In [ ]:
evals, evecs = np.linalg.eigh(Gamma_values[-1])
order = np.argsort(evals)[::-1]
evals, evecs = evals[order], evecs[:, order]
plt.plot(evals, marker='.')
plt.yscale('log')
biggest = evecs[:, :3]
print(evals[:3])
# print(biggest)
print(-np.linalg.inv(biggest.T @ Gamma_values[-1] @ biggest))

In [ ]:
pi @ (bigF - pairs[locs])
#, pairs[locs]

In [ ]:
# np.linalg.norm(Gamma-Gamma.T)/np.linalg.norm(Gamma)

In [ ]:
np.sort(np.linalg.eigvals(Z))[:5]

In [ ]:
bigF.T @ np.diag(pi) @ Z_from_R(pi, R) @ bigF